## Clasificatori de bucătărie 2

În această a doua lecție despre clasificare, vom explora `mai multe modalități` de a clasifica datele categorice. De asemenea, vom învăța despre implicațiile alegerii unui clasificator în detrimentul altuia.

### [**Chestionar înainte de lecție**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Prerechizite**

Presupunem că ați finalizat lecțiile anterioare, deoarece vom continua să folosim unele concepte pe care le-am învățat anterior.

Pentru această lecție, vom avea nevoie de următoarele pachete:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) este o [colecție de pachete R](https://www.tidyverse.org/packages) concepută pentru a face știința datelor mai rapidă, mai ușoară și mai distractivă!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) este un [cadru de lucru](https://www.tidymodels.org/packages/) format din pachete pentru modelare și învățare automată.

-   `themis`: Pachetul [themis](https://themis.tidymodels.org/) oferă pași suplimentari pentru rețete, utili în gestionarea datelor dezechilibrate.

Le puteți instala astfel:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativ, scriptul de mai jos verifică dacă aveți pachetele necesare pentru a finaliza acest modul și le instalează pentru dvs. în cazul în care lipsesc.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. O hartă de clasificare**

În [lecția anterioară](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), am încercat să răspundem la întrebarea: cum alegem între mai multe modele? În mare măsură, acest lucru depinde de caracteristicile datelor și de tipul de problemă pe care dorim să o rezolvăm (de exemplu, clasificare sau regresie?).

Anterior, am învățat despre diversele opțiuni pe care le aveți atunci când clasificați date folosind fișa de ajutor de la Microsoft. Framework-ul de Machine Learning al Python, Scikit-learn, oferă o fișă de ajutor similară, dar mai detaliată, care poate ajuta în continuare la restrângerea alegerii estimatoarelor (un alt termen pentru clasificatori):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Sfat: [vizitează această hartă online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) și navighează pe traseu pentru a citi documentația.
>
> [Site-ul de referință Tidymodels](https://www.tidymodels.org/find/parsnip/#models) oferă, de asemenea, o documentație excelentă despre diferite tipuri de modele.

### **Planul** 🗺️

Această hartă este foarte utilă odată ce ai o înțelegere clară a datelor tale, deoarece poți 'merge' pe traseele ei pentru a lua o decizie:

-   Avem \>50 mostre

-   Vrem să prezicem o categorie

-   Avem date etichetate

-   Avem mai puțin de 100K mostre

-   ✨ Putem alege un Linear SVC

-   Dacă asta nu funcționează, deoarece avem date numerice

    -   Putem încerca un ✨ KNeighbors Classifier

        -   Dacă nici asta nu funcționează, încercăm ✨ SVC și ✨ Ensemble Classifiers

Acesta este un traseu foarte util de urmat. Acum, să trecem direct la treabă folosind cadrul de modelare [tidymodels](https://www.tidymodels.org/): o colecție consistentă și flexibilă de pachete R dezvoltate pentru a încuraja bunele practici statistice 😊.

## 2. Împărțirea datelor și gestionarea setului de date dezechilibrat.

Din lecțiile anterioare, am învățat că există un set de ingrediente comune în bucătăriile noastre. De asemenea, am observat o distribuție destul de inegală în numărul de bucătării.

Vom gestiona aceste aspecte prin:

-   Eliminarea celor mai comune ingrediente care creează confuzie între bucătării distincte, folosind `dplyr::select()`.

-   Utilizarea unui `recipe` care preprocesează datele pentru a le pregăti pentru modelare prin aplicarea unui algoritm de `over-sampling`.

Am discutat deja despre cele de mai sus în lecția anterioară, așa că ar trebui să fie o joacă de copii 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Gestionarea datelor dezechilibrate

Datele dezechilibrate au adesea efecte negative asupra performanței modelului. Multe modele funcționează cel mai bine atunci când numărul de observații este egal și, prin urmare, tind să întâmpine dificultăți cu datele dezechilibrate.

Există în principal două modalități de a gestiona seturile de date dezechilibrate:

-   adăugarea de observații la clasa minoritară: `Supraselectare` (Over-sampling), de exemplu, utilizând un algoritm SMOTE care generează sintetic noi exemple ale clasei minoritare folosind cei mai apropiați vecini ai acestor cazuri.

-   eliminarea observațiilor din clasa majoritară: `Subselectare` (Under-sampling)

În lecția noastră anterioară, am demonstrat cum să gestionăm seturile de date dezechilibrate folosind un `recipe`. Un recipe poate fi considerat ca un plan care descrie ce pași ar trebui aplicați unui set de date pentru a-l pregăti pentru analiza datelor. În cazul nostru, ne dorim să avem o distribuție egală a numărului de bucătării pentru `setul nostru de antrenament`. Haideți să trecem direct la subiect.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Acum suntem gata să antrenăm modele 👩‍💻👨‍💻!

## 3. Dincolo de modelele de regresie multinomială

În lecția anterioară, am analizat modelele de regresie multinomială. Haideți să explorăm câteva modele mai flexibile pentru clasificare.

### Mașini cu Vectori de Suport

În contextul clasificării, `Mașinile cu Vectori de Suport` reprezintă o tehnică de învățare automată care încearcă să găsească un *hiperplan* care separă "cel mai bine" clasele. Să analizăm un exemplu simplu:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ nu separă clasele. H2~ le separă, dar doar cu o marjă mică. H3~ le separă cu o marjă maximă.

#### Clasificator Linear cu Vectori de Suport

Clusteringul cu Vectori de Suport (SVC) face parte din familia de tehnici ML bazate pe mașini cu vectori de suport. În SVC, hiperplanul este ales astfel încât să separe corect `majoritatea` observațiilor de antrenament, dar `poate clasifica greșit` câteva observații. Prin permiterea unor puncte să fie pe partea greșită, SVM devine mai robust la valori extreme, ceea ce duce la o generalizare mai bună pentru datele noi. Parametrul care reglează această abatere este denumit `cost`, având o valoare implicită de 1 (vezi `help("svm_poly")`).

Să creăm un SVC liniar setând `degree = 1` într-un model SVM polinomial.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Acum că am capturat pașii de preprocesare și specificațiile modelului într-un *workflow*, putem continua cu antrenarea SVC liniar și evaluarea rezultatelor în același timp. Pentru măsurile de performanță, să creăm un set de metrici care va evalua: `acuratețea`, `sensibilitatea`, `Valoarea Predictivă Pozitivă` și `F Measure`.

> `augment()` va adăuga coloane pentru predicții la datele furnizate.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Mașina cu Vectori de Suport

Mașina cu vectori de suport (SVM) este o extensie a clasificatorului cu vectori de suport pentru a permite o delimitare non-lineară între clase. În esență, SVM-urile folosesc *trucul kernel* pentru a extinde spațiul caracteristicilor, adaptându-se astfel la relațiile non-lineare dintre clase. O funcție kernel populară și extrem de flexibilă utilizată de SVM-uri este *funcția de bază radială.* Haideți să vedem cum se va descurca pe datele noastre.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Mult mai bine 🤩!

> ✅ Vă rugăm să consultați:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> pentru lecturi suplimentare.

### Clasificatori Nearest Neighbor

Algoritmul *K*-nearest neighbor (KNN) este o metodă în care fiecare observație este prezisă pe baza *similarității* sale cu alte observații.

Haideți să aplicăm unul pe datele noastre.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Se pare că acest model nu funcționează prea bine. Probabil că modificarea argumentelor modelului (vezi `help("nearest_neighbor")`) va îmbunătăți performanța modelului. Asigură-te că încerci acest lucru.

> ✅ Te rog să consulți:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> pentru a afla mai multe despre clasificatorii *K*-Nearest Neighbors.

### Clasificatori de tip ensemble

Algoritmii de tip ensemble funcționează prin combinarea mai multor estimatori de bază pentru a produce un model optim, fie prin:

`bagging`: aplicarea unei *funcții de mediere* asupra unei colecții de modele de bază

`boosting`: construirea unei secvențe de modele care se bazează unul pe altul pentru a îmbunătăți performanța predictivă.

Să începem prin testarea unui model Random Forest, care construiește o colecție mare de arbori de decizie și apoi aplică o funcție de mediere pentru a obține un model general mai bun.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Bravo 👏!

Haide să experimentăm și cu un model Boosted Tree.

Boosted Tree define o metodă de ansamblu care creează o serie de arbori decizionali secvențiali, unde fiecare arbore depinde de rezultatele arborilor anteriori, încercând să reducă treptat eroarea. Se concentrează pe greutățile elementelor clasificate incorect și ajustează potrivirea pentru următorul clasificator pentru a corecta.

Există diferite moduri de a potrivi acest model (vezi `help("boost_tree")`). În acest exemplu, vom potrivi arborii Boosted prin motorul `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Vă rugăm să consultați:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Explorează modelul AdaBoost, care este o alternativă bună la xgboost.
>
> pentru a afla mai multe despre clasificatorii Ensemble.

## 4. Extra - compararea mai multor modele

Am ajustat destul de multe modele în acest laborator 🙌. Poate deveni obositor sau dificil să creezi o mulțime de fluxuri de lucru din diferite seturi de preprocesare și/sau specificații de model și apoi să calculezi metricele de performanță una câte una.

Haideți să vedem dacă putem aborda această problemă prin crearea unei funcții care ajustează o listă de fluxuri de lucru pe setul de antrenament și apoi returnează metricele de performanță bazate pe setul de testare. Vom folosi `map()` și `map_dfr()` din pachetul [purrr](https://purrr.tidyverse.org/) pentru a aplica funcții fiecărui element din listă.

> Funcțiile [`map()`](https://purrr.tidyverse.org/reference/map.html) vă permit să înlocuiți multe bucle for cu un cod care este atât mai concis, cât și mai ușor de citit. Cel mai bun loc pentru a învăța despre funcțiile [`map()`](https://purrr.tidyverse.org/reference/map.html) este capitolul [iteration](http://r4ds.had.co.nz/iteration.html) din R pentru data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) permite utilizatorilor să creeze și să ajusteze cu ușurință un număr mare de modele, dar este conceput în principal pentru a funcționa cu tehnici de resampling, cum ar fi `cross-validation`, o abordare pe care încă nu am acoperit-o.

## **🚀Provocare**

Fiecare dintre aceste tehnici are un număr mare de parametri pe care îi poți ajusta, de exemplu, `cost` în SVM-uri, `neighbors` în KNN, `mtry` (Predictori Selectați Aleatoriu) în Random Forest.

Cercetează parametrii impliciți ai fiecăruia și gândește-te ce ar însemna ajustarea acestor parametri pentru calitatea modelului.

Pentru a afla mai multe despre un model anume și parametrii săi, folosește: `help("model")`, de exemplu, `help("rand_forest")`.

> În practică, de obicei *estimăm* *cele mai bune valori* pentru acești parametri antrenând mai multe modele pe un `set de date simulat` și măsurând cât de bine performează toate aceste modele. Acest proces se numește **tuning**.

### [**Chestionar post-lectură**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Revizuire & Studiu Individual**

Există mulți termeni specifici în aceste lecții, așa că ia-ți un moment pentru a revizui [această listă](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) de terminologie utilă!

#### MULȚUMIRI SPECIALE:

[`Allison Horst`](https://twitter.com/allison_horst/) pentru crearea ilustrațiilor uimitoare care fac R mai prietenos și mai captivant. Găsește mai multe ilustrații în [galeria ei](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) și [Jen Looper](https://www.twitter.com/jenlooper) pentru crearea versiunii originale în Python a acestui modul ♥️

Învățare plăcută,

[Eric](https://twitter.com/ericntay), Ambasador Gold Microsoft Learn Student.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Ilustrație de @allison_horst</figcaption>



---

**Declinarea responsabilității**:  
Acest document a fost tradus utilizând serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să aveți în vedere că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
